In [1]:
import sys
print(sys.executable)

/home/samuel/anaconda3/envs/carnd-term1/bin/python


In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)


def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if (len(img.shape) > 2):
        channel_count = img.shape[2]
        ignore_mask_color = (255,)*channel_count
    else:
        ignore_mask_color = 255
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [6]:
def get_lines(lines, color=[255, 0, 0]):
    right_lines = []
    left_lines = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        y = y2 - y1
        x = x2 - x1
        slope = y / x
        if slope < -0.1:
            right_lines.append(line)
        elif slope > 0.1:
            left_lines.append(line)
    return right_lines, left_lines

In [7]:
def get_points(lines):
    points = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        points.append((x1, y1))
        points.append((x2, y2))
    points = np.array(points)
    return points

In [8]:
def draw_lines(img, right_points, left_points, color=[255, 0, 0], thickness=5):
    
    right_line = cv2.fitLine(np.array(right_points), cv2.DIST_L2, 0, 0.01, 0.01)
    left_line = cv2.fitLine(np.array(left_points), cv2.DIST_L2, 0, 0.01, 0.01)
    
    right_m = right_line[1] / right_line[0]
    
    left_m = left_line[1] / left_line[0]
    y_ini = img.shape[0]
    y_fin = img.shape[0] / 1.6

    
    right_ini_x = ((y_ini - right_line[3]) / right_m) + right_line[2]
    right_fin_x = ((y_fin - right_line[3]) / right_m) + right_line[2]
    
    left_ini_x = ((y_ini - left_line[3]) / left_m) + left_line[2]
    left_fin_x = ((y_fin - left_line[3]) / left_m) + left_line[2]
    
    cv2.line(img, (right_ini_x, y_ini), (right_fin_x, int(y_fin)), color, thickness)
    cv2.line(img, (left_ini_x, y_ini), (left_fin_x, int(y_fin)), color, thickness)

In [9]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    right_lines, left_lines = get_lines(lines)
    right_points = get_points(right_lines)
    left_points = get_points(left_lines)
    draw_lines(line_img, right_points, left_points)
    return line_img


def weighted_img(img, initial_img, alpha=0.8, beta=1., gamma = 0.):
    return cv2.addWeighted(initial_img, alpha, img,  beta, gamma)

In [10]:
initial_img = mpimg.imread("../test_images/solidYellowLeft.jpg")

In [11]:
def pipeline(img):
    img = grayscale(img)
    img = canny(img, 50, 150)
    img = gaussian_blur(img, 5)
    vertices = np.array([[(0, img.shape[0]), 
                      (470, 300),
                      (490, 300),
                      (img.shape[1] + 80, img.shape[0])]])
    masked_img = region_of_interest(img, vertices)
    lines_img = hough_lines(masked_img, 2, np.pi / 180, 50, 10, 250)
    final = weighted_img(lines_img, initial_img)
    return final

In [12]:
final = pipeline(initial_img)

In [13]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
def process_image(initial_img):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    img = grayscale(initial_img)
    img = canny(img, 50, 150)
    img = gaussian_blur(img, 5)
    vertices = np.array([[(0, img.shape[0]), 
                      (470, 300),
                      (490, 300),
                      (img.shape[1] + 80, img.shape[0])]])
    img = region_of_interest(img, vertices)
    img = hough_lines(img, 2, np.pi / 180, 50, 10, 250)
    img = weighted_img(img, initial_img)
    return img

In [23]:
video_capture = cv2.VideoCapture('../test_videos/solidYellowLeft.mp4')
while (video_capture.isOpened()):
    ret, frame = video_capture.read()
    if ret:
        output = process_image(frame)
        cv2.imshow('frame',output)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# Release everything if job is finished
video_capture.release()
cv2.destroyAllWindows()

In [15]:
white_output = '../test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("../test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   6%|▌         | 13/221 [00:00<00:01, 128.15it/s, now=None]

Moviepy - Building video ../test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video ../test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready ../test_videos_output/solidWhiteRight.mp4
CPU times: user 3.21 s, sys: 137 ms, total: 3.34 s
Wall time: 2.47 s


In [17]:
yellow_output = '../test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('../test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   2%|▏         | 14/681 [00:00<00:05, 130.35it/s, now=None]

Moviepy - Building video ../test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video ../test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready ../test_videos_output/solidYellowLeft.mp4
CPU times: user 10.4 s, sys: 311 ms, total: 10.7 s
Wall time: 7.27 s
